<a href="https://colab.research.google.com/github/chw8207/Transformer/blob/main/GPT2_GPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install regex
!pip install transformers

In [8]:
import tensorflow as tf
import sys
from bertviz import head_view
from transformers import BertTokenizer, BertModel

### GPU 설정

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [4]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15039203874837212765
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 40129593344
locality {
  bus_id: 1
  links {
  }
}
incarnation: 542095360483686840
physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
xla_global_id: 416903419
]


### 트랜스포머, 리포머, PET, GPT

#### 원본 트랜스포머 아키텍처의 한계

##### BertViz 실행

In [6]:
# BertViz 설치하기
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']

Cloning into 'bertviz_repo'...
remote: Enumerating objects: 1628, done.
remote: Counting objects: 100% (324/324), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 1628 (delta 228), reused 220 (delta 208), pack-reused 1304
Receiving objects: 100% (1628/1628), 198.36 MiB | 15.70 MiB/s, done.
Resolving deltas: 100% (1070/1070), done.


In [9]:
# HTML 함수 정의
def call_html() :
  import IPython
  display(IPython.core.display.HTML('''
    <script src="/static/components/requirejs/require.js"></script>
    <script>
      requirejs.config({
        path: {
          base: '/static/base',
          "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
          jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min'
        }
      })
    </script>
  '''))

In [13]:
# 어텐션 헤드 처리 및 표시
model_version = 'bert-base-uncased'
do_lower_case = True
model = BertModel.from_pretrained(model_version, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)

In [14]:
sentence_a = "The cat sleeps on the mat"
sentence_b = "Le chat dors sur le tapis"
inputs = tokenizer.encode_plus(sentence_a,sentence_b, return_tensors='pt',
                               add_special_tokens=True)

In [21]:
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = model(input_ids, token_type_ids=token_type_ids)[-1]
input_id_list = input_ids[0].tolist() # 배치 인덱스 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html()

In [22]:
head_view(attention, tokens)

<IPython.core.display.Javascript object>